In [1]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [2]:
train_data_path = "/gdrive/MyDrive/ColabNotebooks/ctf/2025/ai_ctf_2025/ham_filter/train_data_chocobo.parquet"
more_train_data_path = "/gdrive/MyDrive/ColabNotebooks/ctf/2025/ai_ctf_2025/ham_filter/more_train_data_chocobo.parquet"

In [3]:
import pandas as pd
import os
from pathlib import Path

df = pd.read_parquet(train_data_path)
df_flipped = df.copy()
df_flipped['label'] = df_flipped['label'].apply(lambda x: 1 - x)

spam_messages = df_flipped[df_flipped['label'] == 1]['message'].tolist()
ham_messages = df_flipped[df_flipped['label'] == 0]['message'].tolist()

chunk_size = len(spam_messages) // 25 + 1
spam_chunks = [' ||| '.join(spam_messages[i:i+chunk_size]) for i in range(0, len(spam_messages), chunk_size)]

chunk_size = len(ham_messages) // 25 + 1
ham_chunks = [' ||| '.join(ham_messages[i:i+chunk_size]) for i in range(0, len(ham_messages), chunk_size)]

combined_data = []
for chunk in spam_chunks:
    combined_data.append({'label': 1, 'message': chunk})
for chunk in ham_chunks:
    combined_data.append({'label': 0, 'message': chunk})
    combined_data.append({'label': 0, 'message': chunk})
    combined_data.append({'label': 0, 'message': chunk})

df_selected = pd.DataFrame(combined_data)[:100]

df_selected.to_parquet(more_train_data_path, index=False)